## Assignment Week 5

### Problem #1: Math and Extraction with Zero and One-Shot Prompting

#### Zero-shot Prompt

In the activities below prompt engineering approach such as zero-shot, one-shot and COT are demonstrated in simplest form to generate better quality json data from an invoice using OpenAI LLM (chatgp-4o-mini). 

In [1]:
from openai import OpenAI
import json
import os

In [2]:
# get api key from file
with open("../../apikeys/openai-keys.json", "r") as key_file:
    api_key = json.load(key_file)["default_api_key"]
os.environ["OPENAI_API_KEY"] = api_key

In [3]:
# get client for API call
client = OpenAI()

In [4]:
email_text = """Hey Jim,
I'd like to order some new shoes. Please ship the following:
1. Nike Air Jordan I - 2 pair
2. Converse All-Star - 10 pair
3. New Balance 990 - 1 pair
4. Nike Zoom Fly 5 men's red - 2 pair
Please provide sub-totals and grand total cost.
Thanks.
Lance Gentry
123 Main St.
Chelsea, MI 48109
248-229-2229"""

#### Prompt 1

**No example** about how to extract information was provided in this prompt, which is an example of `zero-shot learning`. The prompt do not guide LLM how to extract the information for the required atribute. Here the list of attributes were explicitly mentioned in the promompt to direct the LLM to generate json containing all the attributes needed in the output as mentioned in the asignment requirement. If the attributes have not been specified it will be solely up to the LLM to generate json attributes based on its understanding which  may or may not match the schema of the requirement json mentioned in the assignment. 

In [5]:
context = "you extract information provided in the attribute list from the email body into Json format. \
You can also add additional attributes in the output json when mentioned in additional attribute list"
attribute_list = "attribute list: shoe_agent_name, request_type shoe_brand,\
shoe_model, shoe_quantity, customer_name, customer_street, customer_city,customer_state,customer_zip, customer_phone, \
shoe_price"
additional_attribute_list = "addtional attribute list: shoe_price for each shoe, shoe_subtotal, and grand_total"


In [6]:
response1 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system", 
            "content": f"{context}"
        },
        {
            "role": "user", 
            "content": f"Q:Here is the email body, attibute list and additional attribute list\n \
            email body:{email_text}\n attribute list: {attribute_list}\n additional attribute list: {additional_attribute_list}\n A:"
        }
    ],
)

print(f"Output json:\n {response1.choices[0].message.content}")

Output json:
 ```json
{
  "shoe_agent_name": "Jim",
  "request_type": "order",
  "shoes": [
    {
      "shoe_brand": "Nike",
      "shoe_model": "Air Jordan I",
      "shoe_quantity": 2,
      "shoe_price": null,  // Price not specified
      "shoe_subtotal": null // Subtotal will be calculated based on shoe_price * shoe_quantity
    },
    {
      "shoe_brand": "Converse",
      "shoe_model": "All-Star",
      "shoe_quantity": 10,
      "shoe_price": null,  // Price not specified
      "shoe_subtotal": null // Subtotal will be calculated based on shoe_price * shoe_quantity
    },
    {
      "shoe_brand": "New Balance",
      "shoe_model": "990",
      "shoe_quantity": 1,
      "shoe_price": null,  // Price not specified
      "shoe_subtotal": null // Subtotal will be calculated based on shoe_price * shoe_quantity
    },
    {
      "shoe_brand": "Nike",
      "shoe_model": "Zoom Fly 5 men's red",
      "shoe_quantity": 2,
      "shoe_price": null,  // Price not specified
      "shoe

In [7]:
#show token usage of api call
print(response1.usage)

CompletionUsage(completion_tokens=370, prompt_tokens=231, total_tokens=601, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))


The response of the `prompt 1` is fair. The output json contain **most information** from the email. However it is interesting to notice that the amount attributes such as `shoe_price`,`shoe_subtotal` and `grand_total` are all `null` because the price of the shoes are not present in the email text. It is interesting to note that the response informs the user that it was nt able to calculate the subtotal and grand total due to absence of shoe price. _This can be resolved by providing the shoe price of each type of shoes in the prompt itself._

#### Prompt 2

In this prompt the `price of each type of shoes` is passed to LLM along with all other information from `prompt 1`.

In [9]:
response2 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system", 
            "content": f"{context}"
        },
        {
            "role": "user", 
            "content": f"Q: Here is the email body, attibute list and additional attribute list\n \
            email body:{email_text}\n attribute list: {attribute_list}\n additional attribute list: {additional_attribute_list}\n\
            More information: *price of Nike Air Jordan I is 100*\n*price of Converse All-Star is 30*\n\
            *price of New Balance 990 is 110*\n*price of Nike Zoom Fly 5 is 400*\n A:"
            
        }
    ],
)

print(f"Output json:\n {response2.choices[0].message.content}")

Output json:
 ```json
{
  "shoe_agent_name": "Jim",
  "request_type": "order",
  "shoes": [
    {
      "shoe_brand": "Nike",
      "shoe_model": "Air Jordan I",
      "shoe_quantity": 2,
      "shoe_price": 100,
      "shoe_subtotal": 200
    },
    {
      "shoe_brand": "Converse",
      "shoe_model": "All-Star",
      "shoe_quantity": 10,
      "shoe_price": 30,
      "shoe_subtotal": 300
    },
    {
      "shoe_brand": "New Balance",
      "shoe_model": "990",
      "shoe_quantity": 1,
      "shoe_price": 110,
      "shoe_subtotal": 110
    },
    {
      "shoe_brand": "Nike",
      "shoe_model": "Zoom Fly 5 men's red",
      "shoe_quantity": 2,
      "shoe_price": 400,
      "shoe_subtotal": 800
    }
  ],
  "customer_name": "Lance Gentry",
  "customer_street": "123 Main St.",
  "customer_city": "Chelsea",
  "customer_state": "MI",
  "customer_zip": "48109",
  "customer_phone": "248-229-2229",
  "grand_total": 1410
}
```


In [10]:
#show token usage of api call
print(response2.usage)

CompletionUsage(completion_tokens=291, prompt_tokens=280, total_tokens=571, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))


We observe that the issue of `null` in the amount attributes is fixed as soon as the price of each item is passed in the prompt. However, we notice that the attribute containing the list of shoes is named as `shoes` instead of `items` as required in the [sample output][1] of the assignmnt provided in BB. This may be resolved by trying `one-shot prompting` where we provide an example with email body and corresponding json output which will serve as an example for the LLM to better understnad the schema and gnerate correct json matching the requirement.

[1]:https://cyberactive.bellevue.edu/bbcswebdav/xid-104820287_4

#### One-shot Prompt

In one-shot prompt an example of the task with sample email body and required etracted json output will be provided to the LLM.

In [13]:
output_json_example = {
  "shoe_agent_name": "Jim",
  "request_type": "order",
  "items": [
    {
      "shoe_brand": "Nike",
      "shoe_model": "Air Jordan I",
      "shoe_quantity": 2,
      "shoe_price": 100,
      "shoe_subtotal": 200
    },
    {
      "shoe_brand": "Converse",
      "shoe_model": "All-Star",
      "shoe_quantity": 10,
      "shoe_price": 30,
      "shoe_subtotal": 300
    },
    {
      "shoe_brand": "New Balance",
      "shoe_model": "990",
      "shoe_quantity": 1,
      "shoe_price": 110,
      "shoe_subtotal": 110
    },
    {
      "shoe_brand": "Nike",
      "shoe_model": "Zoom Fly 5 men's red",
      "shoe_quantity": 2,
      "shoe_price": 400,
      "shoe_subtotal": 800
    }
  ],
  "customer_name": "Lance Gentry",
  "customer_street": "123 Main St.",
  "customer_city": "Chelsea",
  "customer_state": "MI",
  "customer_zip": "48109",
  "customer_phone": "248-229-2229",
  "grand_total": 1410
}


In [14]:
context = "you extract information from the email body into Json format."

In [15]:
new_email_text = """Hey Michelle,
I'd like to order some new shoes. Please ship the following:
1. Nike Air Jordan I - 1 pair
2. Converse All-Star - 20 pair
3. New Balance 990 - 2 pair
4. Nike Zoom Fly 5 men's red - 5 pair
Please provide sub-totals and grand total cost.
Thanks.
Artis Gilmore
723 Lexington Blvd.
New York, NY 10001
(503) 484-1029"""

In [16]:
# api call using prompt for the completion task
response3 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system", 
            "content": f"{context}"
        },
        {
            "role": "user", 
            "content": f"Q: {email_text} \n A: {output_json_example}  \
            Q:{new_email_text}\n A:"
            
        }
    ],
)

print(f"Output json:\n {response3.choices[0].message.content}")

Output json:
 ```json
{
  "shoe_agent_name": "Michelle",
  "request_type": "order",
  "items": [
    {
      "shoe_brand": "Nike",
      "shoe_model": "Air Jordan I",
      "shoe_quantity": 1,
      "shoe_price": 100,
      "shoe_subtotal": 100
    },
    {
      "shoe_brand": "Converse",
      "shoe_model": "All-Star",
      "shoe_quantity": 20,
      "shoe_price": 30,
      "shoe_subtotal": 600
    },
    {
      "shoe_brand": "New Balance",
      "shoe_model": "990",
      "shoe_quantity": 2,
      "shoe_price": 110,
      "shoe_subtotal": 220
    },
    {
      "shoe_brand": "Nike",
      "shoe_model": "Zoom Fly 5 men's red",
      "shoe_quantity": 5,
      "shoe_price": 400,
      "shoe_subtotal": 2000
    }
  ],
  "customer_name": "Artis Gilmore",
  "customer_street": "723 Lexington Blvd.",
  "customer_city": "New York",
  "customer_state": "NY",
  "customer_zip": "10001",
  "customer_phone": "(503) 484-1029",
  "grand_total": 2920
}
```


In [17]:
#show token usage of api call
print(response3.usage)

CompletionUsage(completion_tokens=293, prompt_tokens=464, total_tokens=757, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))


In `one-shot prompting` above, we do not need to provide details of the shoe prices and information about the required attributes in json separately in prompt like `zero-shot prompting`. Instead an `example` of `email body` and corresponding expected `json` output were provided in the prompt. The `examples` enabled the LLM to steer it response to the desired output including all the details. The LLM was able to infer the prices of each kind of shoes from the example itself. The output json has all the attributes exactly as mentioned in the example in this case.

It is worth to note that the cost of the task is higher in the `one shot` promting compared to `zero shot` because we needed to provide an entire email body and an entire json example in the prompt which increased the number of input tokens hence more cost. However, even though cost was higher we were able to get more tailored response. Therefore, cost and output performance must be evaluated before choosing the type of prompting for a task.

### Problem #2: Chat Completion for Chain of Thought

In [18]:
cot_prompt = """Q: There were nine computers in the server room. Five more computers were installed each day, from Monday to Thursday. How many computers are now in the server room?
A: There are 4 days from Monday to Thursday. 5 computers were added each day. That means in total 4 * 5 = 20 computers were added. There were 9 computers initially, so now there are 9 + 20 = 29 computers. The answer is 29.
Q: Michael had 58 golf balls. On Tuesday, he lost 23 golf balls. On Wednesday, he lost 2 more. How many golf balls did he have at the end of Wednesday?
A: Michael initially had 58 balls. He lost 23 on Tuesday, so after that he has 58 - 23 = 35 balls. On Wednesday he lost 2 more so now he has 35 - 2 = 33 balls. The answer is 33.
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: She bought 5 bagels for $3 each. This means she spent $15. She has $8 left.
Q: When I was 6 my sister was half my age. Now I'm 70 how old is my sister?"""

In [19]:
# api call using prompt for the completion task
response4 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user", 
            "content": f"{cot_prompt}"
            
        }
    ],
)

print(f"Answer:\n {response4.choices[0].message.content}")

Output json:
 When you were 6 years old, your sister was half your age, which means she was 3 years old at that time. The age difference between you and your sister is 3 years. Now that you are 70, your sister would be 70 - 3 = 67 years old. So, your sister is 67 years old now.


`Chain of thought` is a prompting technique that enables reasoning capabilities in LLM responses by providing reasoning examples in the prompt. As observed above, the response yeilded correct result with the use of `chain of thougt` in promt allowing the LLM to understand the mathematical reasoning and generate correct result. It is worth to note that apart from just the answer, LLM response also includes a step by step explanation about how it arrived at the result.